In [2]:
%pip install --quiet datasets pandas transformers huggingface_hub ipywidgets

from datasets import load_dataset
import pandas as pd

dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])

Note: you may need to restart the kernel to use updated packages.


In [3]:
df.head(10)

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,1,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,2,positive
7,50e14c0bb8,Soooo high,1,neutral
8,e050245fbd,Both of you,1,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,2,positive


AutoTokenizer is a special class in the Huggingface Transformers library. It helps you choose the right tokenizer for your model without knowing the details.

Think of it as a smart assistant that knows which tool to use for the job.

The AutoTokenizer is easy to use. You don’t have to remember which tokenizer goes with which model. It ensures you use the correct tokenizer for the model, reducing errors and improving consistency.

Autotokenizer is flexible. It works with many different models, allowing you to switch models without changing much code.

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from huggingface_hub import login
import os
from dotenv import load_dotenv


# Load environment variables from a .env file
load_dotenv()

# Retrieve the token from the environment variables
token = os.getenv("HF_TOKEN")

try:
    login(token=token)
    print("Logged in to Hugging Face Hub")
except Exception as e:
    print(f"Error during login: {e}")
    print("Please ensure you have a valid Hugging Face token in your .env file.")
    exit(1)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
print("Loaded Tokenizer")
base_model = AutoModelForSequenceClassification.from_pretrained("meta-llama/Llama-3.2-1B", num_labels=3)    
print("Loaded Model")


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Logged in to Hugging Face Hub
Loaded Tokenizer


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded Model


In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}

#### Tokenize and Encode Text

In [5]:
# Setting the pad token. By default, LLaMA models do not have a pad token, so we need to set it manually.
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # padding set make the batches equal to the value set in max_length 
    # truncation set to True to truncate the text if it exceeds max_length
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
# print(tokenized_datasets)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Evaluate

In [6]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer, TrainerCallback
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Custom callback to visualize training progress
class ProgressVisualizationCallback(TrainerCallback):  # Inherit from TrainerCallback
    def __init__(self):
        self.training_logs = []
    
    def on_train_begin(self, args, state, control, **kwargs):
        # Initialize at the beginning of training
        self.training_logs = []
        print("Training started!")
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            self.training_logs.append(logs)
            # Plot every 10 logs to avoid slowing down training
            if len(self.training_logs) % 10 == 0:
                self.visualize_progress(state)
    
    def visualize_progress(self, state):
        clear_output(wait=True)
        
        # Extract metrics
        steps = [log.get('step', i) for i, log in enumerate(self.training_logs) if 'loss' in log]
        loss = [log['loss'] for log in self.training_logs if 'loss' in log]
        lr = [log['learning_rate'] for log in self.training_logs if 'learning_rate' in log]
        
        # Create plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss plot
        ax1.plot(steps, loss, label='Training Loss')
        ax1.set_xlabel('Step')
        ax1.set_ylabel('Loss')
        ax1.set_title('Training Loss')
        ax1.legend()
        ax1.grid(True)
        
        # Learning rate plot
        ax2.plot(steps, lr, label='Learning Rate', color='green')
        ax2.set_xlabel('Step')
        ax2.set_ylabel('Learning Rate')
        ax2.set_title('Learning Rate Schedule')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        # Print current stats
        if steps:
            print(f"Current step: {steps[-1]}/{state.max_steps}")
            print(f"Current loss: {loss[-1]:.4f}")
            print(f"Current learning rate: {lr[-1]:.8f}")
            
            # Calculate training speed
            if len(steps) > 10:
                recent_steps = steps[-10:]
                if len(recent_steps) > 1 and state.max_steps:
                    steps_per_log = recent_steps[1] - recent_steps[0]
                    remaining_steps = state.max_steps - steps[-1]
                    steps_to_go = max(0, remaining_steps)
                    estimated_logs = steps_to_go / steps_per_log if steps_per_log else 0
                    print(f"Approximately {estimated_logs:.0f} logs remaining")

# Setup training with proper logging
training_args = TrainingArguments(
    output_dir="test_trainer",
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,  # Log every 10 steps
    evaluation_strategy="steps",
    eval_steps=100,    # Evaluate every 100 steps
    save_strategy="steps",
    save_steps=100,    # Save model every 100 steps
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,    # Set a specific number of steps
    report_to="none",  # Disable wandb/tensorboard to avoid conflicts
    learning_rate=2e-5,
    weight_decay=0.01,
)

# Create the trainer with our callback
progress_callback = ProgressVisualizationCallback()
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[progress_callback],  # Add the callback here
)

# Start training
print("Starting training...")
trainer.train()

Starting training...
Training started!


Step,Training Loss,Validation Loss
